In [3]:
from encoder.utils import convert_audio
import torchaudio
import torch
from decoder.pretrained import WavTokenizer
import glob

In [4]:
device=torch.device('cpu')

config_path = "../configs/wavtokenizer_smalldata_frame40_3s_nq1_code4096_dim512_kmeans200_attn.yaml"
model_path = "../checkpoints/WavTokenizer_small_600_24k_4096.ckpt"
audio_path = "../data/common_voice_pt/clips/common_voice_pt_19273359.mp3"

wavtokenizer = WavTokenizer.from_pretrained0802(config_path, model_path)
wavtokenizer = wavtokenizer.to(device)

In [5]:
path_to_wavs = '../data/common_voice_pt/clips'
pattern = f'{path_to_wavs}/*.mp3'

wavs_path = glob.glob(pattern)
wavs_path = wavs_path[:5]

In [6]:
for audio_path in wavs_path:
    wav, sr = torchaudio.load(audio_path)
    wav = convert_audio(wav, sr, 24000, 1)
    bandwidth_id = torch.tensor([0])
    wav=wav.to(device)
    features,discrete_code= wavtokenizer.encode_infer(wav, bandwidth_id=bandwidth_id)

    file_name = audio_path.split('/')[-1].split('.')[0]
    code_outpath = '../output/' + file_name + '.pt'
    audio_outpath = '../output/' + file_name + '_infer.wav'
    audio_outpath = '../output/' + file_name + '_orig.wav'

    torch.save(discrete_code, code_outpath)

    bandwidth_id = torch.tensor([0])  
    audio_out = wavtokenizer.decode(features, bandwidth_id=bandwidth_id)

    torchaudio.save(audio_outpath, wav, sample_rate=24000, encoding='PCM_S', bits_per_sample=16)
    torchaudio.save(audio_outpath, audio_out, sample_rate=24000, encoding='PCM_S', bits_per_sample=16)